In [ ]:
import numpy as np
from numpy.linalg import inv
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib import colormaps
from ipywidgets import interact, widgets
import time
import seaborn as sns

# 1. Standard Asset Allocation

#### Read data

In [1]:
# First read all excel sheets
market_cap_daily = pd.read_excel('Group_G.xlsx', sheet_name=1, index_col='Date')
price_index_daily = pd.read_excel('Group_G.xlsx', sheet_name=2, index_col='Date')
revenue_monthly = pd.read_excel('Group_G.xlsx', sheet_name=3, index_col='Date')
emission_monthly = pd.read_excel('Group_G.xlsx', sheet_name=4, index_col='Date')


# # Reformat the column name
market_cap_daily.columns = ['_'.join(i.split('-')[0].split(' ')[:-1]) for i in market_cap_daily.keys()]
price_index_daily.columns = ['_'.join(i.split('-')[0].split(' ')[:-1]) for i in price_index_daily.keys()]
# revenue_monthly.columns = ['_'.join(i.split('-')[0].split(' ')[:2]) for i in revenue_monthly.keys()]
emission_monthly.columns = ['_'.join(i.split('-')[0].split(' ')[:-1]) for i in emission_monthly.keys()]


# Using the data from the last day of the month for market cap and price index
price_index_monthly = price_index_daily.resample('M').last()
market_cap_monthy = market_cap_daily.resample('M').last()

NameError: name 'pd' is not defined

## 1.1 Describe the characteristics of the stocks 

In [24]:
# Get simple and log returns
def assets_returns(assets, period, compounded=True):
    if not compounded:
        asset_return = (assets / assets.shift(periods=period)) - 1
    else:
        diff = assets / assets.shift(periods=period)
        asset_return = np.log(diff.astype('float'))
    return asset_return


# Get moments of returns
def get_moment(asset_return, moment):
    if moment == 1:
        return pd.DataFrame(asset_return.mean())
    elif moment == 2:
        return pd.DataFrame(asset_return.std())
    elif moment == 3:
        return pd.DataFrame(asset_return.skew())
    elif moment == 4:
        nu = (pow(asset_return, moment)).mean() - pow(asset_return.mean(), moment)
        den = pow(asset_return.std(), moment)
        res = nu / den
        return pd.DataFrame(res)

In [25]:
# Compute Returns 

# Simple return 
simple_return = assets_returns(
    price_index_monthly,
    period=1,
    compounded=False  # not compounded
)

# Log return 
log_return = assets_returns(
    price_index_monthly,
    period=1,
    compounded=True  # compounded
)

### Test for distribution of stocks

To test for the distribution of stocks and subsequently using MLE to estimate the in-sample mean, we have the following steps:

1. Use rolling window to get sample moments by year 
2. Use Jarque Bera to test for the normality of returns 
3. Use Smirnov to test for distribution of data 
4. Use MLE to estimate 

In [1]:
sns.displot(log_return.iloc[:,1])

NameError: name 'sns' is not defined

## 1.2 Build the efficient frontier using sample mean and sample covariance matrix

## 1.3 Use resampling to determine optimal weight